In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import re
import json
import h2o

from datetime import datetime

from utils import get_data, Config, clean_url, make_clickable, add_columns

from tree import make_tree

In [3]:
# Load all data (except unknown/manual testing)
# Parsing + Basic 5x+ each, and reruns (such that all tests have 5+ results)
# Ubuntu 22.04, Android 11, MacOS 14.3.1, iPadOS 17.3.1
# All runs performed between 2024-03-04 and 2024-03-NN
initial_data = """
SELECT "Result".*, 
"Response".raw_header, "Response".status_code, "Response".label, "Response".resp_type,
"Browser".name, "Browser".version, "Browser".headless_mode, "Browser".os, "Browser".automation_mode, "Browser".add_info
FROM "Result"
JOIN "Response" ON "Result".response_id = "Response".id JOIN "Browser" ON "Result".browser_id = "Browser".id
WHERE "Browser".name != 'Unknown';
"""
df = get_data(Config(), initial_data)
df = add_columns(df)

Connecting to the PostgreSQL database...
Connection successful


In [4]:
df["raw_header"] = df["raw_header"].apply(bytes).apply(lambda x: json.loads(x.decode("utf-8"))).astype(str)
def create_test_id(row):
    return f'{row["test_name"]}_{row["relation_info"]}_{row["org_scheme"]}_{row["org_host"]}_{row["resp_scheme"]}_{row["resp_host"]}_{row["response_id"]}_{row["resp_type"]}'

df["browser_id"] = df["browser_id"].astype("category")
# Takes a while (1000s+) (might be faster to already do it with postgres but not too important)
df["test_id"] = df.apply(create_test_id, axis=1)
df["test_id"] = df["test_id"].astype("category")

In [6]:
responses = """
SELECT * from "Response";
"""
responses = get_data(Config(), responses)

Connecting to the PostgreSQL database...
Connection successful


# Overview

In [5]:
# Remove other browsers/os
df = df.loc[~df["browser_id"].isin([30, 27, 26, 54, 28])]

In [6]:
# Ignore MacOS and iPad OS for now as they are not finished yet?!
df_m = df
df = df.loc[~df["browser_id"].isin([51, 60])]

In [7]:
# Both basic + parsing mode
# Each test should have results 5+ times (such that we can better reason about noise + majority voting makes sense)
df.groupby(["automation_mode", "browser_id", "name", "version", "os", "headless_mode"], observed=True)["id"].count().sort_values(ascending=False).to_frame().reset_index()

,automation_mode,browser_id,name,version,os,headless_mode,id
0,intent,65,brave,1.62.165,Android 11,real,1100797
1,intent,64,firefox_beta,123.0b9,Android 11,real,1015902
2,intent,62,chrome,121.0.6167.180,Android 11,real,922650
3,intent,63,opera,80.5.4244.78163,Android 11,real,907650
4,selenium,47,chrome,122,Ubuntu 22.04,headless-new,896618
5,selenium,45,brave,v1.62.156 (121.0.6167.139),Ubuntu 22.04,headless-new,896230
6,selenium,49,firefox,121,Ubuntu 22.04,headless,896166
7,selenium,61,firefox,123,Ubuntu 22.04,headless,896166
8,selenium,44,edge,121,Ubuntu 22.04,headless-new,896152
9,selenium,43,firefox,122,Ubuntu 22.04,headless,896150


# Error + Timeout Analysis
- Iterative design:
  - We use higher timeouts for certain tests that timed out more often in debug runs
  - We use higher timeouts for Android
  - We use higher timeouts for the repeat runs
- Small number of random timeouts in both parsing/basic mode
- Very small number of not-run in Opera and Brave Android (HSTS)

In [23]:
# Timeout (2), Fail (1), Not-run (3)
filtered_df = df.groupby(["browser"])["test_status"].filter(lambda x: x.nunique() >= 2)
display(df[["browser", "test_status"]].loc[df.index.isin(filtered_df.index)].groupby(["browser"])["test_status"].value_counts().to_frame())

count
browser                                            test_status         
brave Android 11 1.62.165 intent real              0            1100792
                                                   3                  5
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s... 0             896222
                                                   2                  8
chrome Android 11 121.0.6167.180 intent real       0             922649
                                                   2                  1
chrome Ubuntu 22.04 122 selenium headless-new      0             896601
                                                   2                 17
edge Ubuntu 22.04 121 selenium headless-new        0             896151
                                                   2                  1
firefox Ubuntu 22.04 121 selenium headless         0             896165
                                                   2                  1
firefox Ubuntu 22.04 123 selenium headless         0             896165
                                                   2                  1
firefox_beta Android 11 123.0b9 intent real        0            1015711
                                                   2                191
opera Android 11 80.5.4244.78163 intent real       0             907611
                                                   2                 31
                                                   3                  8

In [24]:
# Teststatus == 2 (timeout)

# Rare timeouts in some browsers
display(df[df["test_status"] == 2]["browser"].value_counts().to_frame())

with pd.option_context("display.max_rows", 80):
    display(df.loc[(df["test_status"] == 2)].groupby(["resp_type", "status_code", "test_name"])["browser"].value_counts().to_frame().sort_values(by="count", ascending=False))

,count
browser,
firefox_beta Android 11 123.0b9 intent real,191
opera Android 11 80.5.4244.78163 intent real,31
chrome Ubuntu 22.04 122 selenium headless-new,17
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,8
chrome Android 11 121.0.6167.180 intent real,1
firefox Ubuntu 22.04 121 selenium headless,1
edge Ubuntu 22.04 121 selenium headless-new,1
firefox Ubuntu 22.04 123 selenium headless,1


count
resp_type status_code test_name               browser                                                  
basic     200         oac_iframe              firefox_beta Android 11 123.0b9 intent real            96
                      referrer_iframe         firefox_beta Android 11 123.0b9 intent real            42
          302         oac_iframe              firefox_beta Android 11 123.0b9 intent real            28
          200         fullscreen_iframe       opera Android 11 80.5.4244.78163 intent real           15
parsing   200         accesswindow_direct     firefox_beta Android 11 123.0b9 intent real            14
                      fullscreen_iframe       opera Android 11 80.5.4244.78163 intent real            9
basic     200         referrer_iframe         brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      6
                      fullscreen_iframe       chrome Ubuntu 22.04 122 selenium headless-new           5
parsing   200         framing_iframe          firefox_beta Android 11 123.0b9 intent real             5
                      script_execution_iframe opera Android 11 80.5.4244.78163 intent real            4
basic     200         framing_object          chrome Ubuntu 22.04 122 selenium headless-new           4
                      framing_embed           chrome Ubuntu 22.04 122 selenium headless-new           4
          302         referrer_iframe         firefox_beta Android 11 123.0b9 intent real             4
          200         framing_iframe          chrome Ubuntu 22.04 122 selenium headless-new           3
                      fullscreen_iframe       brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      2
          302         oac_window.open         firefox_beta Android 11 123.0b9 intent real             2
                      framing_embed           chrome Android 11 121.0.6167.180 intent real            1
          200         referrer_iframe         chrome Ubuntu 22.04 122 selenium headless-new           1
                                              firefox Ubuntu 22.04 121 selenium headless              1
                                              firefox Ubuntu 22.04 123 selenium headless              1
parsing   200         fullscreen_iframe       edge Ubuntu 22.04 121 selenium headless-new             1
          201         script_execution_iframe opera Android 11 80.5.4244.78163 intent real            1
          203         script_execution_iframe opera Android 11 80.5.4244.78163 intent real            1
          204         script_execution_iframe opera Android 11 80.5.4244.78163 intent real            1

In [26]:
# Teststatus == 3 (not-run)

display(df[df["test_status"] == 3]["browser"].value_counts().to_frame())

display(df.loc[(df["test_status"] == 3)].groupby(["resp_type", "status_code", "test_name"])["browser"].value_counts().to_frame().sort_values(by="count", ascending=False))

,count
browser,
opera Android 11 80.5.4244.78163 intent real,8
brave Android 11 1.62.165 intent real,5


count
resp_type status_code test_name             browser                                            
parsing   200         upgradeHSTS_subdomain brave Android 11 1.62.165 intent real             5
                      upgradeHSTS_direct    opera Android 11 80.5.4244.78163 intent real      4
                      upgradeHSTS_subdomain opera Android 11 80.5.4244.78163 intent real      4

# General Test Statistics

In [27]:
# Number of response_ids tested (should be 44415)
df.groupby("browser")["response_id"].nunique()

browser
brave Android 11 1.62.165 intent real                                  44415
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new    44415
chrome Android 11 121.0.6167.180 intent real                           44415
chrome Ubuntu 22.04 120 selenium headless-new                          44415
chrome Ubuntu 22.04 121 selenium headless-new                          44415
chrome Ubuntu 22.04 122 selenium headless-new                          44415
edge Ubuntu 22.04 121 selenium headless-new                            44415
firefox Ubuntu 22.04 121 selenium headless                             44415
firefox Ubuntu 22.04 122 selenium headless                             44415
firefox Ubuntu 22.04 123 selenium headless                             44415
firefox_beta Android 11 123.0b9 intent real                            44415
opera Android 11 80.5.4244.78163 intent real                           44415
Name: response_id, dtype: int64

In [28]:
df.groupby("browser")["full_url"].nunique()

browser
brave Android 11 1.62.165 intent real                                  215094
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new     26115
chrome Android 11 121.0.6167.180 intent real                           142105
chrome Ubuntu 22.04 120 selenium headless-new                           25174
chrome Ubuntu 22.04 121 selenium headless-new                           25183
chrome Ubuntu 22.04 122 selenium headless-new                           28966
edge Ubuntu 22.04 121 selenium headless-new                             25343
firefox Ubuntu 22.04 121 selenium headless                              25116
firefox Ubuntu 22.04 122 selenium headless                              25600
firefox Ubuntu 22.04 123 selenium headless                              25102
firefox_beta Android 11 123.0b9 intent real                            240928
opera Android 11 80.5.4244.78163 intent real                           129279
Name: full_url, dtype: int64

In [29]:
# Number of test_ids
# Should be 10456 and 168774
df.loc[df["test_status"] == 0].groupby(["resp_type", "browser"])["test_id"].nunique()

resp_type  browser                                                            
basic      brave Android 11 1.62.165 intent real                                   10456
           brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new     10456
           chrome Android 11 121.0.6167.180 intent real                            10456
           chrome Ubuntu 22.04 120 selenium headless-new                           10456
           chrome Ubuntu 22.04 121 selenium headless-new                           10456
           chrome Ubuntu 22.04 122 selenium headless-new                           10456
           edge Ubuntu 22.04 121 selenium headless-new                             10456
           firefox Ubuntu 22.04 121 selenium headless                              10456
           firefox Ubuntu 22.04 122 selenium headless                              10456
           firefox Ubuntu 22.04 123 selenium headless                              10456
           firefox_beta Android

In [30]:
# Number of response_ids for each "group": resp_type, test_name, relation_info
df.groupby(["resp_type", "test_name", "relation_info"])["response_id"].nunique().to_frame()

response_id
resp_type test_name                     relation_info              
basic     accesswindow_direct           direct                   11
          fetch_GET                     credentials               6
                                        custom_headers            6
                                        simple                    6
          fetch_TEST                    custom_method             6
          framing_embed                 direct                   35
                                        nested                   35
                                        sandbox                  35
          framing_iframe                direct                   35
                                        nested                   35
                                        sandbox                  35
          framing_object                direct                   35
                                        nested                   35
                                        sandbox                  35
          fullscreen_iframe             child                    24
                                        child_allow              24
                                        child_sandbox            24
                                        direct                   24
          imgloading_iframe             direct                   17
          oac_iframe                    direct                   14
                                        sandbox                  14
          oac_window.open               window.open              14
          perfAPI_img                   direct                   13
          referrer_iframe               iframe                   16
                                        window.open              16
          script_execution_iframe       direct                   17
                                        sandbox                  17
          subresourceloadingCOEP_img    direct                   11
                                        sandbox                  11
          subresourceloadingCORP_img    direct                   11
                                        nested                   11
                                        sandbox                  11
          subresourceloadingCORP_object direct                   11
          upgradeHSTS_direct            direct                   14
          upgradeHSTS_subdomain         subdomain                14
parsing   accesswindow_direct           direct                 2656
          fetch_GET                     credentials            8059
                                        custom_headers         8059
                                        simple                 8059
          fetch_TEST                    custom_method          8059
          framing_iframe                direct                 8780
                                        nested                 8780
          fullscreen_iframe             child_allow            3880
                                        direct                 3880
          imgloading_iframe             direct                 2710
          oac_window.open               window.open            1699
          perfAPI_img                   direct                 1601
          referrer_iframe               iframe                 2797
          script_execution_iframe       direct                 2726
          subresourceloadingCOEP_img    direct                 2637
          subresourceloadingCORP_img    direct                 3357
          upgradeHSTS_direct            direct                 3324
          upgradeHSTS_subdomain         subdomain              3324

# Stability
- How many tests have more than one result for the 5+ repetitions in a browser?
- Basic:
    - total of 273/(10456*12) have different results
    - subresourceloadingCOEP_img is the worst offender (especially random behavior Firefox bug)
    - should be mostly timeout related and "fixed" by majority voting
- Parsing:
    - total of 966/(168774*12) have different results
    - subresourceloadingCOEP_img + hsts
    - ...

In [31]:
# Remove all tests that timed out (test_status == 0), each test should have at least 5 results that did not time out in each browser
df_org = df
df = df.loc[df["test_status"] == 0]

In [32]:
details = False
for group_name, resp_type_group in df.groupby("resp_type"):
    print(group_name)
    stab = resp_type_group.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser"])[["outcome_str"]].agg(["count", "nunique"])    # Tests with more than one outcome
    diff_outcomes = stab.loc[stab[("outcome_str", "nunique")] != 1]
    print(f"{len(diff_outcomes)} tests have different outcomes!")
    display(diff_outcomes.reset_index()["browser"].value_counts().to_frame())
    with pd.option_context("display.max_rows", 100):
        display(diff_outcomes.reset_index()[["test_name", "browser"]].value_counts().to_frame())

    diff_with_responses = diff_outcomes.reset_index().groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "browser"])["response_id"].agg(list).to_frame().reset_index()

    if not details:
        continue
    for id, (test_name, org_scheme, org_host, resp_scheme, resp_host, relation_info, browser, response_ids) in diff_with_responses.iterrows():
        print(test_name, response_ids)
        rows = df.loc[(df["org_scheme"] == org_scheme) & (df["test_name"] == test_name) & (df["resp_scheme"] == resp_scheme) & (df["resp_host"] == resp_host) & (df["relation_info"] == relation_info) & (df["response_id"].isin(response_ids)) & (df["browser"] == browser)]
        with pd.option_context("display.max_colwidth", 200):
            display(rows[["outcome_str", "browser"]].value_counts().to_frame())
            disp = rows.drop_duplicates(subset="outcome_str")[["browser", "outcome_str", "raw_header", "test_name", "org_scheme", "org_host", "resp_scheme", "relation_info", "resp_host", "full_url"]]
            disp = disp.style.format({'full_url': make_clickable})
            display(disp)

basic
273 tests have different outcomes!


,count
browser,
firefox_beta Android 11 123.0b9 intent real,95
opera Android 11 80.5.4244.78163 intent real,39
firefox Ubuntu 22.04 123 selenium headless,35
firefox Ubuntu 22.04 121 selenium headless,32
firefox Ubuntu 22.04 122 selenium headless,31
brave Android 11 1.62.165 intent real,19
chrome Android 11 121.0.6167.180 intent real,8
chrome Ubuntu 22.04 120 selenium headless-new,8
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,5


count
(test_name, )              (browser, )                                              
subresourceloadingCOEP_img firefox_beta Android 11 123.0b9 intent real            95
                           firefox Ubuntu 22.04 123 selenium headless             34
                           firefox Ubuntu 22.04 121 selenium headless             32
                           firefox Ubuntu 22.04 122 selenium headless             30
referrer_iframe            opera Android 11 80.5.4244.78163 intent real           17
accesswindow_direct        opera Android 11 80.5.4244.78163 intent real           12
oac_window.open            opera Android 11 80.5.4244.78163 intent real           10
                           brave Android 11 1.62.165 intent real                   9
perfAPI_img                chrome Ubuntu 22.04 120 selenium headless-new           7
referrer_iframe            brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      5
                           brave Android 11 1.62.165 intent real                   4
upgradeHSTS_direct         brave Android 11 1.62.165 intent real                   4
accesswindow_direct        chrome Android 11 121.0.6167.180 intent real            3
referrer_iframe            chrome Android 11 121.0.6167.180 intent real            3
upgradeHSTS_subdomain      brave Android 11 1.62.165 intent real                   2
oac_window.open            chrome Android 11 121.0.6167.180 intent real            2
framing_object             firefox Ubuntu 22.04 122 selenium headless              1
framing_embed              firefox Ubuntu 22.04 123 selenium headless              1
referrer_iframe            chrome Ubuntu 22.04 120 selenium headless-new           1
                           chrome Ubuntu 22.04 121 selenium headless-new           1

parsing
966 tests have different outcomes!


,count
browser,
firefox Ubuntu 22.04 123 selenium headless,121
brave Android 11 1.62.165 intent real,119
chrome Ubuntu 22.04 122 selenium headless-new,113
chrome Ubuntu 22.04 121 selenium headless-new,112
chrome Android 11 121.0.6167.180 intent real,97
chrome Ubuntu 22.04 120 selenium headless-new,83
opera Android 11 80.5.4244.78163 intent real,78
firefox Ubuntu 22.04 122 selenium headless,76
firefox Ubuntu 22.04 121 selenium headless,59


count
(test_name, )              (browser, )                                              
subresourceloadingCOEP_img chrome Ubuntu 22.04 121 selenium headless-new          65
                           firefox Ubuntu 22.04 121 selenium headless             56
upgradeHSTS_direct         chrome Ubuntu 22.04 120 selenium headless-new          55
                           brave Android 11 1.62.165 intent real                  54
subresourceloadingCOEP_img firefox Ubuntu 22.04 123 selenium headless             53
upgradeHSTS_direct         firefox Ubuntu 22.04 123 selenium headless             49
subresourceloadingCOEP_img firefox Ubuntu 22.04 122 selenium headless             48
                           firefox_beta Android 11 123.0b9 intent real            45
upgradeHSTS_direct         chrome Android 11 121.0.6167.180 intent real           44
subresourceloadingCOEP_img chrome Ubuntu 22.04 122 selenium headless-new          44
imgloading_iframe          chrome Ubuntu 22.04 121 selenium headless-new          42
subresourceloadingCOEP_img edge Ubuntu 22.04 121 selenium headless-new            24
upgradeHSTS_direct         opera Android 11 80.5.4244.78163 intent real           21
framing_iframe             opera Android 11 80.5.4244.78163 intent real           19
subresourceloadingCOEP_img chrome Android 11 121.0.6167.180 intent real           19
upgradeHSTS_subdomain      firefox Ubuntu 22.04 123 selenium headless             19
framing_iframe             chrome Ubuntu 22.04 122 selenium headless-new          19
upgradeHSTS_subdomain      brave Android 11 1.62.165 intent real                  18
subresourceloadingCOEP_img chrome Ubuntu 22.04 120 selenium headless-new          17
upgradeHSTS_subdomain      chrome Ubuntu 22.04 122 selenium headless-new          15
subresourceloadingCOEP_img brave Android 11 1.62.165 intent real                  14
script_execution_iframe    firefox Ubuntu 22.04 122 selenium headless             14
                           brave Android 11 1.62.165 intent real                  13
upgradeHSTS_subdomain      chrome Android 11 121.0.6167.180 intent real           13
                           opera Android 11 80.5.4244.78163 intent real           12
upgradeHSTS_direct         chrome Ubuntu 22.04 122 selenium headless-new          12
imgloading_iframe          brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...     11
fetch_GET                  opera Android 11 80.5.4244.78163 intent real           10
imgloading_iframe          firefox Ubuntu 22.04 122 selenium headless             10
perfAPI_img                chrome Ubuntu 22.04 122 selenium headless-new          10
fetch_GET                  brave Android 11 1.62.165 intent real                  10
script_execution_iframe    opera Android 11 80.5.4244.78163 intent real            9
imgloading_iframe          chrome Ubuntu 22.04 120 selenium headless-new           9
accesswindow_direct        brave Android 11 1.62.165 intent real                   8
script_execution_iframe    chrome Android 11 121.0.6167.180 intent real            8
imgloading_iframe          chrome Ubuntu 22.04 122 selenium headless-new           7
fetch_GET                  brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      7
imgloading_iframe          chrome Android 11 121.0.6167.180 intent real            6
upgradeHSTS_direct         edge Ubuntu 22.04 121 selenium headless-new             5
subresourceloadingCOEP_img brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      5
script_execution_iframe    chrome Ubuntu 22.04 122 selenium headless-new           4
accesswindow_direct        chrome Android 11 121.0.6167.180 intent real            4
subresourceloadingCOEP_img opera Android 11 80.5.4244.78163 intent real            3
script_execution_iframe    firefox Ubuntu 22.04 121 selenium headless              3
upgradeHSTS_direct         firefox_beta Android 11 123.0b9 intent real             2
fullscreen_iframe          opera Android 11 80.5.4244.78163 intent real            2

## More stability?!
- How many outcomes did we collect per test in the different browsers? Also shows tests with only one outcome
- How many of them are different?

In [33]:
# How many outcomes did we collect for each test?!
# Should be 5+ (less than 5 is concerning as we cannot do proper majority voting in such cases)

final_df = pd.DataFrame()

for group, d in df.groupby("browser"):
    # Calculate counts and unique counts
    aggs = ["count"] # ["count", "nunique"]
    # d = d.loc[d["org_scheme"] == "http"]
    res = d.groupby(["test_id"], observed=True)["outcome_str"].agg(aggs)
    # Get value counts for each combination of counts and unique counts
    counts = res.value_counts().to_frame()    
    counts = counts.rename(columns={"count": group})
    display(counts)
    final_df = pd.concat([final_df, counts], axis=1)

display(final_df)

,brave Android 11 1.62.165 intent real
count,
5,163424
24,1657
6,1598
9,1191
8,1014
7,965
10,963
14,927
34,718


,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new
count,
5,179158
6,72


,chrome Android 11 121.0.6167.180 intent real
count,
5,173460
14,1477
6,1142
10,874
7,854
8,623
9,583
13,169
12,22


,chrome Ubuntu 22.04 120 selenium headless-new
count,
5,179230


,chrome Ubuntu 22.04 121 selenium headless-new
count,
5,179230


...

,firefox Ubuntu 22.04 122 selenium headless
count,
5,179230


,firefox Ubuntu 22.04 123 selenium headless
count,
5,179215
6,15


,firefox_beta Android 11 123.0b9 intent real
count,
5,174067
20,836
21,812
14,510
50,455
19,435
22,353
13,281
51,260


,opera Android 11 80.5.4244.78163 intent real
count,
5,176904
10,1692
9,391
13,113
6,55
14,30
12,15
8,15
7,10


,brave Android 11 1.62.165 intent real,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,chrome Android 11 121.0.6167.180 intent real,chrome Ubuntu 22.04 120 selenium headless-new,chrome Ubuntu 22.04 121 selenium headless-new,chrome Ubuntu 22.04 122 selenium headless-new,edge Ubuntu 22.04 121 selenium headless-new,firefox Ubuntu 22.04 121 selenium headless,firefox Ubuntu 22.04 122 selenium headless,firefox Ubuntu 22.04 123 selenium headless,firefox_beta Android 11 123.0b9 intent real,opera Android 11 80.5.4244.78163 intent real
count,,,,,,,,,,,,
5,163424.0,179158.0,173460.0,179230.0,179230.0,178779.0,179229.0,179215.0,179230.0,179215.0,174067.0,176904.0
24,1657.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
6,1598.0,72.0,1142.0,NaN,NaN,451.0,1.0,15.0,NaN,15.0,7.0,55.0
9,1191.0,NaN,583.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,391.0
8,1014.0,NaN,623.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,15.0
7,965.0,NaN,854.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,10.0
10,963.0,NaN,874.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,1692.0
14,927.0,NaN,1477.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,510.0,30.0
34,718.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN


In [34]:
# Which tests have how many outcomes?

grouped = df.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser", "resp_type"])['outcome_str'].count().reset_index()

with pd.option_context("display.max_rows", 256):
    res = grouped.groupby(["browser", "resp_type", "test_name"])["outcome_str"].agg(["mean", "min", "max", "count", "sum"]).sort_values(by="mean", ascending=False)
    display(res.describe())
    display(res.loc[res["mean"] > 0].head(20))
    display(res.loc[res["mean"] > 0].tail(20))


,mean,min,max,count,sum
count,384.000000,384.0,384.000000,384.000000,384.000000
mean,5.149826,5.0,8.161458,5600.937500,28949.263021
std,0.843122,0.0,9.229874,10294.362861,53335.589445
min,5.000000,5.0,5.000000,28.000000,140.000000
25%,5.000000,5.0,5.000000,284.000000,1430.000000
50%,5.000000,5.0,5.000000,1680.000000,8400.000000
75%,5.007879,5.0,6.000000,5487.500000,27970.000000
max,16.491760,5.0,54.000000,48354.000000,342579.000000


mean  \
browser                                      resp_type test_name                            
firefox_beta Android 11 123.0b9 intent real  parsing   oac_window.open          16.491760   
                                                       accesswindow_direct      12.941453   
brave Android 11 1.62.165 intent real        parsing   script_execution_iframe  10.913426   
firefox_beta Android 11 123.0b9 intent real  parsing   imgloading_iframe         9.383395   
brave Android 11 1.62.165 intent real        parsing   oac_window.open           7.273396   
                                                       fetch_GET                 7.084812   
                                                       fetch_TEST                7.084812   
chrome Android 11 121.0.6167.180 intent real parsing   oac_window.open           6.691583   
opera Android 11 80.5.4244.78163 intent real basic     accesswindow_direct       6.522727   
firefox_beta Android 11 123.0b9 intent real  basic     oac_iframe                6.473214   
chrome Android 11 121.0.6167.180 intent real parsing   accesswindow_direct       6.429593   
brave Android 11 1.62.165 intent real        basic     upgradeHSTS_subdomain     6.357143   
                                                       upgradeHSTS_direct        6.357143   
                                             parsing   accesswindow_direct       5.725527   
firefox_beta Android 11 123.0b9 intent real  parsing   fullscreen_iframe         5.663402   
                                             basic     oac_window.open           5.656250   
brave Android 11 1.62.165 intent real        basic     perfAPI_img               5.653846   
                                             parsing   framing_iframe            5.617938   
chrome Android 11 121.0.6167.180 intent real basic     upgradeHSTS_direct        5.571429   
                                                       upgradeHSTS_subdomain     5.571429   

                                                                                min  \
browser                                      resp_type test_name                      
firefox_beta Android 11 123.0b9 intent real  parsing   oac_window.open            5   
                                                       accesswindow_direct        5   
brave Android 11 1.62.165 intent real        parsing   script_execution_iframe    5   
firefox_beta Android 11 123.0b9 intent real  parsing   imgloading_iframe          5   
brave Android 11 1.62.165 intent real        parsing   oac_window.open            5   
                                                       fetch_GET                  5   
                                                       fetch_TEST                 5   
chrome Android 11 121.0.6167.180 intent real parsing   oac_window.open            5   
opera Android 11 80.5.4244.78163 intent real basic     accesswindow_direct        5   
firefox_beta Android 11 123.0b9 intent real  basic     oac_iframe                 5   
chrome Android 11 121.0.6167.180 intent real parsing   accesswindow_direct        5   
brave Android 11 1.62.165 intent real        basic     upgradeHSTS_subdomain      5   
                                                       upgradeHSTS_direct         5   
                                             parsing   accesswindow_direct        5   
firefox_beta Android 11 123.0b9 intent real  parsing   fullscreen_iframe          5   
                                             basic     oac_window.open            5   
brave Android 11 1.62.165 intent real        basic     perfAPI_img                5   
                                             parsing   framing_iframe             5   
chrome Android 11 121.0.6167.180 intent real basic     upgradeHSTS_direct         5   
                                                       upgradeHSTS_subdomain      5   

                                                                                max  \
browser                             

mean  \
browser                                       resp_type test_name                             
chrome Ubuntu 22.04 122 selenium headless-new basic     fetch_TEST                      5.0   
                                                        fetch_GET                       5.0   
                                                        accesswindow_direct             5.0   
chrome Ubuntu 22.04 121 selenium headless-new parsing   upgradeHSTS_subdomain           5.0   
                                                        upgradeHSTS_direct              5.0   
                                                        subresourceloadingCORP_img      5.0   
                                                        subresourceloadingCOEP_img      5.0   
                                                        script_execution_iframe         5.0   
                                              basic     subresourceloadingCORP_object   5.0   
                                              parsing   perfAPI_img                     5.0   
                                                        oac_window.open                 5.0   
                                                        imgloading_iframe               5.0   
                                                        fullscreen_iframe               5.0   
                                                        framing_iframe                  5.0   
                                                        fetch_TEST                      5.0   
                                                        fetch_GET                       5.0   
                                                        accesswindow_direct             5.0   
                                              basic     upgradeHSTS_subdomain           5.0   
                                                        upgradeHSTS_direct              5.0   
                                              parsing   referrer_iframe                 5.0   

                                                                                       min  \
browser                                       resp_type test_name                            
chrome Ubuntu 22.04 122 selenium headless-new basic     fetch_TEST                       5   
                                                        fetch_GET                        5   
                                                        accesswindow_direct              5   
chrome Ubuntu 22.04 121 selenium headless-new parsing   upgradeHSTS_subdomain            5   
                                                        upgradeHSTS_direct               5   
                                                        subresourceloadingCORP_img       5   
                                                        subresourceloadingCOEP_img       5   
                                                        script_execution_iframe          5   
                                              basic     subresourceloadingCORP_object    5   
                                              parsing   perfAPI_img                      5   
                                                        oac_window.open                  5   
                                                        imgloading_iframe                5   
                                                        fullscreen_iframe                5   
                                                        framing_iframe                   5   
                                                        fetch_TEST                       5   
                                                        fetch_GET                        5   
                                                        accesswindow_direct              5   
                                              basic     upgradeHSTS_subdomain            5   
                                                        upgradeHSTS_direct               5   
                      

In [35]:
# Different outcomes (in percentage for a test group (test_name)
# (Does not take into account how often each test was executed)
grouped = df.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser", "resp_type"])['outcome_str'].nunique().reset_index()

grouped["diff_outcome"] = grouped["outcome_str"] > 1
with pd.option_context("display.max_rows", 100):
    res = grouped.groupby(["browser", "resp_type", "test_name"])["diff_outcome"].agg(["mean", "count", "sum"]).sort_values(by="mean", ascending=False)
    display(res.describe())
    display(res.loc[res["mean"] > 0].head(20))

,mean,count,sum
count,384.000000,384.000000,384.000000
mean,0.003098,5600.937500,3.226562
std,0.018537,10294.362861,10.564398
min,0.000000,28.000000,0.000000
25%,0.000000,284.000000,0.000000
50%,0.000000,1680.000000,0.000000
75%,0.000000,5487.500000,0.000000
max,0.269886,48354.000000,95.000000


mean  \
browser                                            resp_type test_name                              
firefox_beta Android 11 123.0b9 intent real        basic     subresourceloadingCOEP_img  0.269886   
brave Android 11 1.62.165 intent real              basic     upgradeHSTS_direct          0.142857   
firefox Ubuntu 22.04 123 selenium headless         basic     subresourceloadingCOEP_img  0.096591   
firefox Ubuntu 22.04 121 selenium headless         basic     subresourceloadingCOEP_img  0.090909   
firefox Ubuntu 22.04 122 selenium headless         basic     subresourceloadingCOEP_img  0.085227   
brave Android 11 1.62.165 intent real              basic     upgradeHSTS_subdomain       0.071429   
opera Android 11 80.5.4244.78163 intent real       basic     accesswindow_direct         0.068182   
                                                             oac_window.open             0.044643   
brave Android 11 1.62.165 intent real              basic     oac_window.open             0.040179   
chrome Ubuntu 22.04 120 selenium headless-new      basic     perfAPI_img                 0.033654   
opera Android 11 80.5.4244.78163 intent real       basic     referrer_iframe             0.033203   
chrome Android 11 121.0.6167.180 intent real       basic     accesswindow_direct         0.017045   
chrome Ubuntu 22.04 121 selenium headless-new      parsing   subresourceloadingCOEP_img  0.012325   
firefox Ubuntu 22.04 121 selenium headless         parsing   subresourceloadingCOEP_img  0.010618   
firefox Ubuntu 22.04 123 selenium headless         parsing   subresourceloadingCOEP_img  0.010049   
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s... basic     referrer_iframe             0.009766   
firefox Ubuntu 22.04 122 selenium headless         parsing   subresourceloadingCOEP_img  0.009101   
chrome Android 11 121.0.6167.180 intent real       basic     oac_window.open             0.008929   
firefox_beta Android 11 123.0b9 intent real        parsing   subresourceloadingCOEP_img  0.008532   
chrome Ubuntu 22.04 122 selenium headless-new      parsing   subresourceloadingCOEP_img  0.008343   

                                                                                         count  \
browser                                            resp_type test_name                           
firefox_beta Android 11 123.0b9 intent real        basic     subresourceloadingCOEP_img    352   
brave Android 11 1.62.165 intent real              basic     upgradeHSTS_direct             28   
firefox Ubuntu 22.04 123 selenium headless         basic     subresourceloadingCOEP_img    352   
firefox Ubuntu 22.04 121 selenium headless         basic     subresourceloadingCOEP_img    352   
firefox Ubuntu 22.04 122 selenium headless         basic     subresourceloadingCOEP_img    352   
brave Android 11 1.62.165 intent real              basic     upgradeHSTS_subdomain          28   
opera Android 11 80.5.4244.78163 intent real       basic     accesswindow_direct           176   
                                                             oac_window.open               224   
brave Android 11 1.62.165 intent real              basic     oac_window.open               224   
chrome Ubuntu 22.04 120 selenium headless-new      basic     perfAPI_img                   208   
opera Android 11 80.5.4244.78163 intent real       basic     referrer_iframe               512   
chrome Android 11 121.0.6167.180 intent real       basic     accesswindow_direct           176   
chrome Ubuntu 22.04 121 selenium headless-new      parsing   subresourceloadingCOEP_img   5274   
firefox Ubuntu 22.04 121 selenium headless         parsing   subresourceloadingCOEP_img   5274   
firefox Ubuntu 22.04 123 selenium headless         parsing   subresourceloadingCOEP_img   5274   
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s... basic     referrer_iframe               512   
firefox Ubuntu 22.04 122 selenium headless         parsing   subresourceloadingCOEP_img   5274   
chrome A

# Browser differences!
- First perform majority voting
- Then diff the browsers

In [36]:
df = df_org
df = df.loc[df["test_status"] == 0]

In [37]:
df_stab = df

# Only keep one row for each test
# If the test had non-deterministic results, use majority voting! (only works reliably if there a enough repetitions of the tests runs, >= 5)

print("Original data entries", len(df))

# Majority voting (pd.Series.mode returns the most frequest item) (quite slow)

df = df.groupby(["test_id", "browser"], observed=True)["outcome_str"].agg(pd.Series.mode).reset_index()
print("Only one row for each test (per browser; majority voting)", len(df))

Original data entries 11116517
Only one row for each test (per browser; majority voting) 2150760


In [38]:
# Merge back additional required properties!
# ["name", "test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "resp_type", "label", 'org_origin', 'resp_origin', 'status_code', 'raw_header']

df = df.merge(df_stab.drop_duplicates(subset=["test_id", "browser"]), on=["test_id", "browser"], how="left", suffixes=["", "_ignore"])


In [39]:
# If several values occur the same, the mode is an np.ndarray and not a string!
# If we have exactly 5 repetitions for each test, the issue cannot exist!
# However, our repeat system only enables us to have 5+ results for each test
print("Tests with the same mode (highest frequency outcome):")
display(df.loc[df['outcome_str'].apply(lambda x: isinstance(x, np.ndarray))].groupby(["browser", "resp_type", "test_name"])["outcome_str"].count())

# In the rare cases, where it actually matters, we can just take the first? (this might be noise in the generated trees and other summaries!)
# These are unstable tests that might indicate random behavior in the browser (e.g., firefox COEP_img)
df['outcome_str'] = df['outcome_str'].apply(lambda x: x[0] if isinstance(x, np.ndarray) else x)

Tests with the same mode (highest frequency outcome):


browser                                       resp_type  test_name                 
brave Android 11 1.62.165 intent real         basic      oac_window.open               1
firefox_beta Android 11 123.0b9 intent real   basic      subresourceloadingCOEP_img    8
opera Android 11 80.5.4244.78163 intent real  parsing    upgradeHSTS_direct            1
Name: outcome_str, dtype: int64

In [40]:
# Most tests have exactly one outcome, quite some have two, couple have three!
res = df.groupby(["resp_type", "test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "test_id"], observed=True)
res = res["outcome_str"].agg("nunique")
display(res.value_counts().to_frame())

,count
outcome_str,
1,172167
2,7029
3,34


In [41]:
# Basic:
# subresourceloadingCORP_object has different results for 100% of tests
# couple of others have 20%+ differences
# couple have 0%

# Parsing:
# oac_window.open has 99%
# perfAPI_img has 60% (might be an issue with the current macOS VPN based test run?)
# Other test groups have between 0.5 - 5% of tests with different outcomes
r = res.reset_index()
r["diff"] = r["outcome_str"] != 1
r.groupby(["resp_type", "test_name", "relation_info"])["diff"].agg(["count", "sum", "mean"]).sort_values("mean", ascending=False)

count   sum      mean
resp_type test_name                     relation_info                        
parsing   oac_window.open               window.open      3398  3370  0.991760
basic     oac_window.open               window.open       224   192  0.857143
          oac_iframe                    sandbox           224   144  0.642857
                                        direct            224   144  0.642857
          subresourceloadingCOEP_img    direct            176    86  0.488636
          referrer_iframe               window.open       256   120  0.468750
          fullscreen_iframe             direct            384   142  0.369792
                                        child_allow       384   113  0.294271
          perfAPI_img                   direct            208    52  0.250000
          subresourceloadingCORP_object direct            176    32  0.181818
          accesswindow_direct           direct            176    28  0.159091
          referrer_iframe               iframe            256    18  0.070312
          fullscreen_iframe             child             384    23  0.059896
          imgloading_iframe             direct            272    13  0.047794
parsing   fullscreen_iframe             child_allow      7760   355  0.045747
          imgloading_iframe             direct           5420   230  0.042435
          script_execution_iframe       direct           5452   216  0.039618
          upgradeHSTS_direct            direct           6648   252  0.037906
          fullscreen_iframe             direct           7760   272  0.035052
          framing_iframe                direct          17560   365  0.020786
          referrer_iframe               iframe           5594   101  0.018055
basic     framing_iframe                sandbox           560     8  0.014286
parsing   subresourceloadingCOEP_img    direct           5274    71  0.013462
          upgradeHSTS_subdomain         subdomain        6648    69  0.010379
          subresourceloadingCORP_img    direct           6714    56  0.008341
          framing_iframe                nested          17560   144  0.008200
          accesswindow_direct           direct           5312    40  0.007530
          fetch_GET                     simple          16118   104  0.006452
                                        custom_headers  16118   101  0.006266
          fetch_TEST                    custom_method   16118   101  0.006266
basic     subresourceloadingCORP_img    direct            176     1  0.005682
parsing   fetch_GET                     credentials     16118    86  0.005336
          perfAPI_img                   direct           3202    14  0.004372
basic     fetch_GET                     credentials        96     0  0.000000
                                        custom_headers     96     0  0.000000
                                        simple             96     0  0.000000
          fetch_TEST                    custom_method      96     0  0.000000
          framing_object                sandbox           560     0  0.000000
          fullscreen_iframe             child_sandbox     384     0  0.000000
          framing_embed                 sandbox           560     0  0.000000
                                        nested            560     0  0.000000
          framing_iframe                direct            560     0  0.000000
                                        nested            560     0  0.000000
          framing_object                direct            560     0  0.000000
                                        nested            560     0  0.000000
          framing_embed                 direct            560     0  0.000000
          upgradeHSTS_direct            direct             28     0  0.000000
          subresourceloadingCORP_img    nested            176     0  0.000000
          upgradeHSTS_subdomain         subdomain          28     0  0.000000
          script_execution_iframe       sandbox           272     0  0.000000


In [42]:
# Select only test rows with more than one outcome
tests_with_mult_outcomes = r.loc[r["diff"]].set_index(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])
df_mult = df.loc[df[["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"]].apply(tuple, axis=1).isin(tests_with_mult_outcomes.index)]
tests_with_more_than_one_outcome = df_mult.drop_duplicates(subset=["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])


all_tests_should_be =  df_org.loc[df_org["test_status"] == 0]["test_id"].nunique() * df_org["browser"].nunique() # (num parsing tests + num_basic_tests) * num_browsers (168774+10456)
print(f"All test rows: {len(df)}, should be: {all_tests_should_be}, Rows with more than one outcome: {len(df_mult)}, Tests with more than one outcome: {len(tests_with_more_than_one_outcome)}")

All test rows: 2150760, should be: 2150760, Rows with more than one outcome: 84756, Tests with more than one outcome: 7063


In [43]:
# Display difference groups
for grouping, group in df_mult.groupby(["test_name", "relation_info"]):
    print(grouping)
    df_list = []
    for _, one_test in group.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"]):
        browsers = one_test.groupby("outcome_str")["browser"].unique().apply(list).apply(sorted).to_frame()
        new = True
        for df_b in df_list:
            if df_b.equals(browsers):
                new = False
        if new:
            df_list.append(browsers)
    print(len(df_list))
    for df_b in df_list:
        with pd.option_context("display.max_colwidth", 1000):
            display(df_b)
    #input("Continue!")

('accesswindow_direct', 'direct')
7


,browser
outcome_str,
{'window.open.opener': 'null'},"[brave Android 11 1.62.165 intent real, brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Android 11 121.0.6167.180 intent real, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new]"
"{'window.open.opener': 'object ""[object Window]""'}","[firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless, firefox Ubuntu 22.04 123 selenium headless, firefox_beta Android 11 123.0b9 intent real, opera Android 11 80.5.4244.78163 intent real]"


,browser
outcome_str,
{'window.open.opener': 'null'},"[brave Android 11 1.62.165 intent real, brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Android 11 121.0.6167.180 intent real, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, opera Android 11 80.5.4244.78163 intent real]"
"{'window.open.opener': 'object ""[object Window]""'}","[firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless, firefox Ubuntu 22.04 123 selenium headless, firefox_beta Android 11 123.0b9 intent real]"


,browser
outcome_str,
{'window.open.opener': 'null'},"[brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless, firefox Ubuntu 22.04 123 selenium headless, opera Android 11 80.5.4244.78163 intent real]"
"{'window.open.opener': 'object ""[object Window]""'}","[brave Android 11 1.62.165 intent real, chrome Android 11 121.0.6167.180 intent real, firefox_beta Android 11 123.0b9 intent real]"


,browser
outcome_str,
{'window.open.opener': 'null'},"[firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless, firefox Ubuntu 22.04 123 selenium headless, firefox_beta Android 11 123.0b9 intent real]"
"{'window.open.opener': 'object ""[object Window]""'}","[brave Android 11 1.62.165 intent real, brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Android 11 121.0.6167.180 intent real, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, opera Android 11 80.5.4244.78163 intent real]"


...

,browser
outcome_str,
Fetch timed out,"[firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless, firefox Ubuntu 22.04 123 selenium headless, firefox_beta Android 11 123.0b9 intent real]"
{'response.redirected': False},"[brave Android 11 1.62.165 intent real, brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Android 11 121.0.6167.180 intent real, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, opera Android 11 80.5.4244.78163 intent real]"


,browser
outcome_str,
{'response.redirected': False},"[firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless, firefox Ubuntu 22.04 123 selenium headless, firefox_beta Android 11 123.0b9 intent real]"
{'response.redirected': True},"[brave Android 11 1.62.165 intent real, brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Android 11 121.0.6167.180 intent real, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, opera Android 11 80.5.4244.78163 intent real]"


,browser
outcome_str,
{'response.redirected': False},"[brave Android 11 1.62.165 intent real, brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Android 11 121.0.6167.180 intent real, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, opera Android 11 80.5.4244.78163 intent real]"
{'response.redirected': True},"[firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless, firefox Ubuntu 22.04 123 selenium headless, firefox_beta Android 11 123.0b9 intent real]"


,browser
outcome_str,
{'response.redirected': False},"[brave Android 11 1.62.165 intent real, brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Android 11 121.0.6167.180 intent real, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless, firefox Ubuntu 22.04 123 selenium headless, firefox_beta Android 11 123.0b9 intent real]"
{'response.redirected': True},[opera Android 11 80.5.4244.78163 intent real]


,browser
outcome_str,
Fetch timed out,"[firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless, firefox Ubuntu 22.04 123 selenium headless, firefox_beta Android 11 123.0b9 intent real]"
{'response.redirected': True},"[brave Android 11 1.62.165 intent real, brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Android 11 121.0.6167.180 intent real, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, opera Android 11 80.5.4244.78163 intent real]"


In [44]:
# Other approach to list the difference groups
def get_uniques(df):
    browsers = f"{df['browser'].unique().tolist()}"
    try:
        outcome = unique_outcomes[df.name[0]]
    except KeyError:
        outcome = {}
        #outcome = []
    outcome[browsers] = df.name[1]
    #outcome.append(f"{df.name[1]} ({browsers})")
    unique_outcomes[df.name[0]] = outcome

for grouping, group in df_mult.groupby(["test_name", "relation_info"]):
    print(grouping)
    unique_outcomes = {}
    group.groupby(["browser"])["outcome_str"].value_counts(normalize=False).reset_index().groupby(["outcome_str", "count"], group_keys=True).apply(get_uniques)
    d = pd.DataFrame.from_dict(unique_outcomes, orient="index")
    if d.shape[1] != 1:
        display(d)
        print()

('accesswindow_direct', 'direct')


,['firefox_beta Android 11 123.0b9 intent real'],"['firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless', 'firefox Ubuntu 22.04 123 selenium headless']",['brave Android 11 1.62.165 intent real'],['chrome Android 11 121.0.6167.180 intent real'],['opera Android 11 80.5.4244.78163 intent real'],"['brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new']"
{'window.open.opener': 'null'},12,24,41,45,49,56
"{'window.open.opener': 'object ""[object Window]""'}",56,44,27,23,19,12



('fetch_GET', 'credentials')


,"['brave Android 11 1.62.165 intent real', 'brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Android 11 121.0.6167.180 intent real', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new', 'opera Android 11 80.5.4244.78163 intent real']","['firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless', 'firefox Ubuntu 22.04 123 selenium headless', 'firefox_beta Android 11 123.0b9 intent real']"
"{'error': 'null', 'headers': 'content-length,'}",3.0,10
"{'error': 'null', 'headers': 'content-length,test,'}",19.0,20
"{'error': 'object ""TypeError: Failed to fetch""', 'headers': ''}",64.0,16
"{'error': 'object ""AbortError: Fetch is aborted""', 'headers': ''}",NaN,40



('fetch_GET', 'custom_headers')


...

,"['firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless', 'firefox Ubuntu 22.04 123 selenium headless', 'firefox_beta Android 11 123.0b9 intent real']","['brave Android 11 1.62.165 intent real', 'brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Android 11 121.0.6167.180 intent real', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new', 'opera Android 11 80.5.4244.78163 intent real']"
error,32.0,NaN
load,NaN,32.0



('upgradeHSTS_direct', 'direct')


,['opera Android 11 80.5.4244.78163 intent real'],['brave Android 11 1.62.165 intent real'],"['firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless', 'firefox Ubuntu 22.04 123 selenium headless', 'firefox_beta Android 11 123.0b9 intent real']","['brave Android 11 1.62.165 intent real', 'brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Android 11 121.0.6167.180 intent real', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new', 'opera Android 11 80.5.4244.78163 intent real']","['brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new']",['chrome Android 11 121.0.6167.180 intent real'],"['firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless', 'firefox Ubuntu 22.04 123 selenium headless']",['firefox_beta Android 11 123.0b9 intent real'],"['brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new', 'opera Android 11 80.5.4244.78163 intent real']"
Fetch timed out,1.0,5.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN
{'response.redirected': False},20.0,19.0,NaN,NaN,21.0,22.0,233.0,235.0,NaN
{'response.redirected': True},NaN,210.0,NaN,NaN,NaN,212.0,7.0,5.0,213.0
TypeError: Failed to fetch,NaN,NaN,NaN,18.0,NaN,NaN,NaN,NaN,NaN



('upgradeHSTS_subdomain', 'subdomain')


,"['firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless', 'firefox Ubuntu 22.04 123 selenium headless', 'firefox_beta Android 11 123.0b9 intent real']","['brave Android 11 1.62.165 intent real', 'brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Android 11 121.0.6167.180 intent real', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new', 'opera Android 11 80.5.4244.78163 intent real']",['opera Android 11 80.5.4244.78163 intent real'],"['brave Android 11 1.62.165 intent real', 'brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Android 11 121.0.6167.180 intent real', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new']"
Fetch timed out,12.0,NaN,NaN,NaN
{'response.redirected': False},54.0,NaN,14.0,15.0
{'response.redirected': True},3.0,NaN,37.0,36.0
TypeError: Failed to fetch,NaN,18.0,NaN,NaN


In [45]:
# Simmilarity between browsers and versions!
# (Current issue: displays a difference if one browser has no result at all; in the end we have 5+ stable results for each tests, thus it should not be an issue anymore)

# Function to calculate absolute and percentage overlap
def calculate_overlap(df, col1, col2):
    df = df[[col1, col2]].dropna()
    df.columns.values[0] = 0
    df.columns.values[1] = 1
    absolute_overlap = (df[0] == df[1]).sum()
    unique_rows = len(df)
    diff_rows = unique_rows - absolute_overlap
    percentage_diff = (diff_rows / unique_rows) * 100
    return diff_rows, percentage_diff, unique_rows


def display_overlap(df, show_all=True, name="All"):
    print(name)
    sim_frame = df.set_index(["test_id", "browser"])["outcome_str"].unstack().astype("str")
    sim_frame = sim_frame.reset_index().drop(columns=["test_id"])
    # Create a dictionary to store results
    results = {}
    matrix = {}
    
    # Iterate through all pairs of columns
    for i, col1 in enumerate(sim_frame.columns):
        for col2 in sim_frame.columns:
            key = f"{col1}_vs_{col2}"
            results[key] = calculate_overlap(sim_frame, col1, col2)
            try:
                l = matrix[col1]
            except KeyError:
                l = {}
            # 0 for absolute count of differences, 1 for percentage
            l[col2] = results[key][0]
            matrix[col1] = l
            
    
    # Convert the results to a DataFrame for better visualization
    results_df = pd.DataFrame.from_dict(results, orient='index', columns=['Absolute Diff', 'Percentage Diff', "Count"])
    
    # Display difference matrix
    with pd.option_context("display.max_columns", 28):
        if show_all:
            # TODO: use a custom Order (e.g., all Android, or some chromium versions next to each other)
            display(pd.DataFrame(matrix))
        else:
            display(pd.DataFrame(matrix).drop_duplicates().T)


display_overlap(df)

for name, group in df.groupby("resp_type"):
    display_overlap(group, show_all=True, name=name)

for name, group in df.groupby("test_name"):
    display_overlap(group, show_all=False, name=name)

All


,brave Android 11 1.62.165 intent real,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,chrome Android 11 121.0.6167.180 intent real,chrome Ubuntu 22.04 120 selenium headless-new,chrome Ubuntu 22.04 121 selenium headless-new,chrome Ubuntu 22.04 122 selenium headless-new,edge Ubuntu 22.04 121 selenium headless-new,firefox Ubuntu 22.04 121 selenium headless,firefox Ubuntu 22.04 122 selenium headless,firefox Ubuntu 22.04 123 selenium headless,firefox_beta Android 11 123.0b9 intent real,opera Android 11 80.5.4244.78163 intent real
brave Android 11 1.62.165 intent real,0,152,82,101,102,102,102,6963,6960,6958,6971,125
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,152,0,161,141,142,142,142,6896,6893,6891,6930,182
chrome Android 11 121.0.6167.180 intent real,82,161,0,20,21,21,21,6931,6928,6926,6939,50
chrome Ubuntu 22.04 120 selenium headless-new,101,141,20,0,1,1,1,6923,6920,6918,6957,46
chrome Ubuntu 22.04 121 selenium headless-new,102,142,21,1,0,0,0,6924,6921,6919,6958,47
chrome Ubuntu 22.04 122 selenium headless-new,102,142,21,1,0,0,0,6924,6921,6919,6958,47
edge Ubuntu 22.04 121 selenium headless-new,102,142,21,1,0,0,0,6924,6921,6919,6958,47
firefox Ubuntu 22.04 121 selenium headless,6963,6896,6931,6923,6924,6924,6924,0,27,33,90,6899
firefox Ubuntu 22.04 122 selenium headless,6960,6893,6928,6920,6921,6921,6921,27,0,38,83,6896
firefox Ubuntu 22.04 123 selenium headless,6958,6891,6926,6918,6919,6919,6919,33,38,0,87,6894


basic


,brave Android 11 1.62.165 intent real,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,chrome Android 11 121.0.6167.180 intent real,chrome Ubuntu 22.04 120 selenium headless-new,chrome Ubuntu 22.04 121 selenium headless-new,chrome Ubuntu 22.04 122 selenium headless-new,edge Ubuntu 22.04 121 selenium headless-new,firefox Ubuntu 22.04 121 selenium headless,firefox Ubuntu 22.04 122 selenium headless,firefox Ubuntu 22.04 123 selenium headless,firefox_beta Android 11 123.0b9 intent real,opera Android 11 80.5.4244.78163 intent real
brave Android 11 1.62.165 intent real,0,64,77,83,84,84,84,1036,1036,1037,1060,107
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,64,0,77,71,72,72,72,1055,1055,1056,1079,106
chrome Android 11 121.0.6167.180 intent real,77,77,0,6,7,7,7,1007,1007,1008,1031,34
chrome Ubuntu 22.04 120 selenium headless-new,83,71,6,0,1,1,1,1013,1013,1014,1037,40
chrome Ubuntu 22.04 121 selenium headless-new,84,72,7,1,0,0,0,1014,1014,1015,1038,41
chrome Ubuntu 22.04 122 selenium headless-new,84,72,7,1,0,0,0,1014,1014,1015,1038,41
edge Ubuntu 22.04 121 selenium headless-new,84,72,7,1,0,0,0,1014,1014,1015,1038,41
firefox Ubuntu 22.04 121 selenium headless,1036,1055,1007,1013,1014,1014,1014,0,12,15,54,983
firefox Ubuntu 22.04 122 selenium headless,1036,1055,1007,1013,1014,1014,1014,12,0,15,54,983
firefox Ubuntu 22.04 123 selenium headless,1037,1056,1008,1014,1015,1015,1015,15,15,0,51,984


parsing


...

,brave Android 11 1.62.165 intent real,firefox Ubuntu 22.04 121 selenium headless
brave Android 11 1.62.165 intent real,0,32
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,0,32
chrome Android 11 121.0.6167.180 intent real,0,32
chrome Ubuntu 22.04 120 selenium headless-new,0,32
chrome Ubuntu 22.04 121 selenium headless-new,0,32
chrome Ubuntu 22.04 122 selenium headless-new,0,32
edge Ubuntu 22.04 121 selenium headless-new,0,32
firefox Ubuntu 22.04 121 selenium headless,32,0
firefox Ubuntu 22.04 122 selenium headless,32,0
firefox Ubuntu 22.04 123 selenium headless,32,0


upgradeHSTS_direct


,brave Android 11 1.62.165 intent real,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,chrome Android 11 121.0.6167.180 intent real,firefox Ubuntu 22.04 121 selenium headless,firefox_beta Android 11 123.0b9 intent real,opera Android 11 80.5.4244.78163 intent real
brave Android 11 1.62.165 intent real,0,7,5,250,248,6
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,7,0,3,244,246,5
chrome Android 11 121.0.6167.180 intent real,5,3,0,247,245,4
chrome Ubuntu 22.04 120 selenium headless-new,7,0,3,244,246,5
chrome Ubuntu 22.04 121 selenium headless-new,7,0,3,244,246,5
chrome Ubuntu 22.04 122 selenium headless-new,7,0,3,244,246,5
edge Ubuntu 22.04 121 selenium headless-new,7,0,3,244,246,5
firefox Ubuntu 22.04 121 selenium headless,250,244,247,0,2,249
firefox Ubuntu 22.04 122 selenium headless,250,244,247,0,2,249
firefox Ubuntu 22.04 123 selenium headless,250,244,247,0,2,249


upgradeHSTS_subdomain


,brave Android 11 1.62.165 intent real,firefox Ubuntu 22.04 121 selenium headless,opera Android 11 80.5.4244.78163 intent real
brave Android 11 1.62.165 intent real,0,68,1
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,0,68,1
chrome Android 11 121.0.6167.180 intent real,0,68,1
chrome Ubuntu 22.04 120 selenium headless-new,0,68,1
chrome Ubuntu 22.04 121 selenium headless-new,0,68,1
chrome Ubuntu 22.04 122 selenium headless-new,0,68,1
edge Ubuntu 22.04 121 selenium headless-new,0,68,1
firefox Ubuntu 22.04 121 selenium headless,68,0,69
firefox Ubuntu 22.04 122 selenium headless,68,0,69
firefox Ubuntu 22.04 123 selenium headless,68,0,69


In [46]:
# Limit on responses that have more than one outcome!
# This will remove all the branches that are the same in all browsers (and make trees large and confusing)
condition = df.groupby(["test_id"], observed=True)["outcome_str"].transform("nunique") != 1
tree_df = df.loc[condition]
print("Remove tests that are the same in all browsers", len(tree_df))

Remove tests that are the same in all browsers 84756


In [47]:
# How many rows exist for tree creation
with pd.option_context("display.max_rows", 100):
    display(tree_df[["test_name", "label", "relation_info"]].value_counts())

test_name                      label       relation_info 
oac_window.open                OAC         window.open       42744
fullscreen_iframe              PP          child_allow        5616
                                           direct             4968
upgradeHSTS_direct             HSTS        direct             3024
imgloading_iframe              CSP-IMG     direct             2916
framing_iframe                 CSP-FA      direct             2628
script_execution_iframe        CSP-SCRIPT  direct             2592
subresourceloadingCOEP_img     COEP        direct             1884
oac_iframe                     OAC         sandbox            1728
                                           direct             1728
referrer_iframe                RP          window.open        1440
                                           iframe             1428
framing_iframe                 CSP-FA      nested             1284
                               XFO         direct              888
    

In [40]:
h2o.init(nthreads=50, max_mem_size="100G", log_level="WARN")
h2o.no_progress()

base_dir = f"trees/{datetime.today().strftime('%Y-%m-%dT%H:%M')}" 

for group_name, group in tree_df.groupby(["test_name", "relation_info"]):
    print(group_name)
    pred_cols = ["browser", "org_origin", "resp_origin", "status_code", "raw_header"]
    group = group[["outcome_str", *pred_cols]]
    tree = make_tree(group, pred_cols, group_name, base_dir)
    # print(tree)


Checking whether there is an H2O instance running at http://localhost:54321. connected.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,20 days 1 hour 6 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,4 months and 5 days
H2O_cluster_name:,H2O_from_python_ubuntu_u7y7iq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,99.9 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


('accesswindow_direct', 'direct')
Connecting to H2O server at http://localhost:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,20 days 1 hour 6 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,4 months and 5 days
H2O_cluster_name:,H2O_from_python_ubuntu_u7y7iq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,99.9 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


Create tree: ('accesswindow_direct', 'direct'), datapoints: 748
('fetch_GET', 'credentials')
Connecting to H2O server at http://localhost:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


...

Create tree: ('subresourceloadingCORP_object', 'direct'), datapoints: 352
('upgradeHSTS_direct', 'direct')
Connecting to H2O server at http://localhost:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,20 days 1 hour 6 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,4 months and 5 days
H2O_cluster_name:,H2O_from_python_ubuntu_u7y7iq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,99.8 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


Create tree: ('upgradeHSTS_direct', 'direct'), datapoints: 2761
('upgradeHSTS_subdomain', 'subdomain')
Connecting to H2O server at http://localhost:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,20 days 1 hour 6 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,4 months and 5 days
H2O_cluster_name:,H2O_from_python_ubuntu_u7y7iq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,99.8 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


Create tree: ('upgradeHSTS_subdomain', 'subdomain'), datapoints: 759


# Manual analysis

In [377]:
# Chromium-based treats host-source as case sensitive even though it should not? (URLs and CSP is case-insensitive!)
# Play around: https://observer.sectec.rocks/opg/iframe/?url=https://echo.sectec.rocks/echo/?content-security-policy=img-src%20HTTPS://echo.sectec.rocks&ecocnt_css=%3Cimg%20src=https://echo.sectec.rocks/%3E&content-type=text/html
# Issue is in the scheme match which should be case-insensitive but is in Chromium-based browsers https://w3c.github.io/webappsec-csp/#match-schemes
# host-part matching was fixed here https://issues.chromium.org/issues/41412675, seems like they forgot fixing scheme-sensitivity 
df.loc[df["raw_header"].str.contains("img-src HTTPS://")][["clean_url", "raw_header", "outcome_str", "browser"]].value_counts().to_frame()

count
clean_url                                          raw_header                                         outcome_str browser                                                  
http://sub.headers.websec.saarland/_hp/tests/su... [['content-security-policy', 'img-src HTTPS://S... error       brave Android 11 1.60.116 intent real                   1
                                                                                                                  brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      1
                                                                                                                  chrome Android 11 120.0.6099.44 intent real             1
                                                                                                                  chrome Ubuntu 22.04 120 selenium headless-new           1
                                                                                                                  chrome Ubuntu 22.04 121 selenium headless-new           1
                                                                                                                  chrome Ubuntu 22.04 122 selenium headless-new           1
                                                                                                                  chrome iPadOS 17.3.1 122.0.6261.89 intent real          1
                                                                                                                  edge Ubuntu 22.04 121 selenium headless-new             1
                                                                                                                  firefox Ubuntu 22.04 121 selenium headless              1
                                                                                                                  firefox Ubuntu 22.04 122 selenium headless              1
                                                                                                                  firefox_beta Android 11 122.0b1 intent real             1
                                                                                                                  opera Android 11 78.4.4143.75735 intent real            1
                                                                                                                  safari macOS 14.3.1 17.3.1 selenium real                1
https://sub.headers.websec.saarland/_hp/tests/s... [['content-security-policy', 'img-src HTTPS://S... error       brave Android 11 1.60.116 intent real                   1
                                                                                                                  brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      1
                                                                                                                  chrome Android 11 120.0.6099.44 intent real             1
                                                                                                                  chrome Ubuntu 22.04 120 selenium headless-new           1
                                                                                                                  chrome Ubuntu 22.04 121 selenium headless-new           1
                                                                                                                  chrome Ubuntu 22.04 122 selenium headless-new           1
                                                                                                                  edge Ubuntu 22.04 121 selenium headless-new             1
                                                                                                                  opera Android 11 78.4.4143.75735 intent real            1
                                                                                                      load        chrome iPadOS 17.3.1 122.0.6261.89 intent real          1
                                      

In [378]:
bf = df.set_index(["test_id", "browser"])["outcome_str"].unstack().astype("str")

def com_browsers(df, b1, b2):
    bf["cr"] = bf[b1] != bf[b2]
    with pd.option_context("display.max_colwidth", None):
        display(bf.loc[bf["cr"]][[b1, b2]])

In [379]:
# Chromium 122 vs 121 vs 120 (desktop linux)
com_browsers(bf, "chrome Ubuntu 22.04 120 selenium headless-new", "chrome Ubuntu 22.04 121 selenium headless-new")
com_browsers(bf, "chrome Ubuntu 22.04 120 selenium headless-new", "chrome Ubuntu 22.04 122 selenium headless-new")
df.loc[df["response_id"] == 40058]["raw_header"].iloc[0]

browser,chrome Ubuntu 22.04 120 selenium headless-new,chrome Ubuntu 22.04 121 selenium headless-new
test_id,,
referrer_iframe_iframe_https_sub.headers.websec.saarland_https_headers.webappsec.eu_40058_parsing,document.referrer: https://headers.webappsec.eu/,document.referrer:
referrer_iframe_window.open_http_sub.headers.websec.saarland_http_sub.headers.websec.saarland_196_basic,document.referrer: full_url,document.referrer: http://sub.headers.websec.saarland/


browser,chrome Ubuntu 22.04 120 selenium headless-new,chrome Ubuntu 22.04 122 selenium headless-new
test_id,,
referrer_iframe_iframe_https_sub.headers.websec.saarland_https_headers.webappsec.eu_40058_parsing,document.referrer: https://headers.webappsec.eu/,document.referrer:
referrer_iframe_window.open_http_sub.headers.websec.saarland_http_sub.headers.websec.saarland_196_basic,document.referrer: full_url,document.referrer: http://sub.headers.websec.saarland/


"[['referrer-policy', 'no-referrer']]"

In [380]:
# Firefox 120 vs 121 (desktop linux)
com_browsers(bf, "firefox Ubuntu 22.04 121 selenium headless", "firefox Ubuntu 22.04 122 selenium headless")
df.loc[df["response_id"] == 65]["raw_header"].iloc[0]

browser,firefox Ubuntu 22.04 121 selenium headless,firefox Ubuntu 22.04 122 selenium headless
test_id,,
subresourceloadingCOEP_img_direct_http_sub.headers.websec.saarland_http_headers.websec.saarland_61_basic,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}","{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}"
subresourceloadingCOEP_img_direct_http_sub.headers.websec.saarland_http_sub.headers.websec.saarland_61_basic,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}","{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}"
subresourceloadingCOEP_img_direct_http_sub.headers.websec.saarland_http_sub.sub.headers.websec.saarland_61_basic,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}","{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}"
subresourceloadingCOEP_img_direct_https_sub.headers.websec.saarland_https_headers.webappsec.eu_18975_parsing,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}","{'image-events': {'swag.jpg': 'error', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}"
subresourceloadingCOEP_img_direct_https_sub.headers.websec.saarland_https_headers.webappsec.eu_18977_parsing,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}","{'image-events': {'swag.jpg': 'error', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}"
subresourceloadingCOEP_img_direct_https_sub.headers.websec.saarland_https_headers.webappsec.eu_18978_parsing,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}","{'image-events': {'swag.jpg': 'error', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}"
subresourceloadingCOEP_img_direct_https_sub.headers.websec.saarland_https_headers.webappsec.eu_20261_parsing,"{'image-events': {'swag.jpg': 'error', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}","{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}"
subresourceloadingCOEP_img_direct_https_sub.headers.websec.saarland_https_headers.webappsec.eu_20262_parsing,"{'image-events': {'swag.jpg': 'error', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}","{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}"
subresourceloadingCOEP_img_direct_https_sub.headers.websec.saarland_https_headers.webappsec.eu_20263_parsing,"{'image-events': {'swag.jpg': 'error', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOr

'[]'

In [381]:
# Brave 121 vs Chrome 121
with pd.option_context("display.max_rows", 72):
    com_browsers(bf, "brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new", "chrome Ubuntu 22.04 121 selenium headless-new")

browser,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,chrome Ubuntu 22.04 121 selenium headless-new
test_id,,
referrer_iframe_iframe_http_sub.headers.websec.saarland_http_headers.webappsec.eu_192_basic,document.referrer: http://headers.webappsec.eu/,document.referrer: full_url
referrer_iframe_iframe_http_sub.headers.websec.saarland_http_headers.webappsec.eu_198_basic,document.referrer: http://headers.webappsec.eu/,document.referrer: full_url
referrer_iframe_iframe_http_sub.headers.websec.saarland_http_headers.webappsec.eu_200_basic,document.referrer: http://headers.webappsec.eu/,document.referrer: full_url
referrer_iframe_iframe_http_sub.headers.websec.saarland_http_headers.websec.saarland_192_basic,document.referrer: http://headers.websec.saarland/,document.referrer: full_url
referrer_iframe_iframe_http_sub.headers.websec.saarland_http_headers.websec.saarland_198_basic,document.referrer: http://headers.websec.saarland/,document.referrer: full_url
...,...,...
referrer_iframe_window.open_https_sub.headers.websec.saarland_https_headers.websec.saarland_198_basic,document.referrer: https://headers.websec.saarland/,document.referrer: full_url
referrer_iframe_window.open_https_sub.headers.websec.saarland_https_headers.websec.saarland_200_basic,document.referrer: https://headers.websec.saarland/,document.referrer: full_url
referrer_iframe_window.open_https_sub.headers.websec.saarland_https_sub.sub.headers.websec.saarland_192_basic,document.referrer: https://sub.sub.headers.websec.saarland/,document.referrer: full_url


In [384]:
with pd.option_context("display.max_rows", 72):
    com_browsers(bf, "chrome iPadOS 17.3.1 122.0.6261.89 intent real", "safari macOS 14.3.1 17.3.1 selenium real")

browser,chrome iPadOS 17.3.1 122.0.6261.89 intent real,safari macOS 14.3.1 17.3.1 selenium real
test_id,,
accesswindow_direct_direct_http_sub.headers.websec.saarland_https_headers.webappsec.eu_21613_parsing,nan,"{'window.open.opener': 'object ""[object Window]""'}"
accesswindow_direct_direct_http_sub.headers.websec.saarland_https_headers.webappsec.eu_21615_parsing,nan,"{'window.open.opener': 'object ""[object Window]""'}"
accesswindow_direct_direct_http_sub.headers.websec.saarland_https_headers.webappsec.eu_21617_parsing,nan,"{'window.open.opener': 'object ""[object Window]""'}"
accesswindow_direct_direct_http_sub.headers.websec.saarland_https_headers.webappsec.eu_21619_parsing,nan,"{'window.open.opener': 'object ""[object Window]""'}"
accesswindow_direct_direct_http_sub.headers.websec.saarland_https_headers.webappsec.eu_21621_parsing,nan,"{'window.open.opener': 'object ""[object Window]""'}"
...,...,...
subresourceloadingCOEP_img_sandbox_http_sub.headers.websec.saarland_http_headers.webappsec.eu_59_basic,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}","{'image-events': {'swag.jpg': 'error', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}"
subresourceloadingCOEP_img_sandbox_http_sub.headers.websec.saarland_http_headers.websec.saarland_59_basic,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}","{'image-events': {'swag.jpg': 'error', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}"
subresourceloadingCOEP_img_sandbox_http_sub.headers.websec.saarland_http_sub.headers.websec.saarland_59_basic,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}","{'image-events': {'swag.jpg': 'error', 'swag-same-site.jpg': 'error', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}"


In [385]:
pivot_df = df.pivot_table(index='test_name', columns='browser', values='test_id', aggfunc='count', fill_value=0)
# Calculate the difference between the counts of the first and second columns
pivot_df['Diff'] = pivot_df.iloc[:, 0] - pivot_df.iloc[:, 1]
display(pivot_df)

browser,brave Android 11 1.60.116 intent real,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,chrome Android 11 120.0.6099.44 intent real,chrome Ubuntu 22.04 120 selenium headless-new,chrome Ubuntu 22.04 121 selenium headless-new,chrome Ubuntu 22.04 122 selenium headless-new,chrome iPadOS 17.3.1 122.0.6261.89 intent real,edge Ubuntu 22.04 121 selenium headless-new,firefox Ubuntu 22.04 121 selenium headless,firefox Ubuntu 22.04 122 selenium headless,firefox_beta Android 11 122.0b1 intent real,opera Android 11 78.4.4143.75735 intent real,safari macOS 14.3.1 17.3.1 selenium real,Diff
test_name,,,,,,,,,,,,,,
accesswindow_direct,5488,5488,5488,5488,5488,5488,4635,5488,5488,5488,5488,5488,5488,0
fetch_GET,48642,48642,48642,48642,48642,48642,48642,48642,48642,48642,48642,48642,48618,0
fetch_TEST,16214,16214,16214,16214,16214,16214,16214,16214,16214,16214,16214,16214,16206,0
framing_embed,1680,1680,1680,1680,1680,1680,1680,1680,1680,1680,1680,1680,1680,0
framing_iframe,36800,36800,36800,36800,36800,36800,36800,36800,36800,36800,36800,36800,36800,0
framing_object,1680,1680,1680,1680,1680,1680,1680,1680,1680,1680,1680,1680,1680,0
fullscreen_iframe,17056,17056,17056,17056,17056,17056,17056,17056,17056,17056,17056,17056,17056,0
imgloading_iframe,5692,5692,5692,5692,5692,5692,5692,5692,5692,5692,5692,5692,5692,0
oac_iframe,448,448,448,448,448,448,448,448,448,448,448,448,448,0
